In [ ]:
import urllib.request
import os
import zipfile
import glob
import numpy as np
import random
from keras import models
from keras import layers
from keras import callbacks
from keras.utils import plot_model
import matplotlib.pyplot as plt
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from keras.models import Sequential
import keras.utils as ku 

# set seeds for reproducability
import tensorflow as tf
from numpy.random import seed


tf.compat.v1.set_random_seed (2)
seed(1)

import pandas as pd
import numpy as np
import string, os 

import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)

Using TensorFlow backend.


In [ ]:
!pip install progressbar
!pip install indic_transliteration

import progressbar
from indic_transliteration import sanscript

  Created wheel for progressbar: filename=progressbar-2.5-cp36-none-any.whl size=12074 sha256=8b83c9d9b5ae82db5197463cb31f267f66232763be98d1ee8d3fc6ef0e2c0e34
  Stored in directory: /root/.cache/pip/wheels/c0/e9/6b/ea01090205e285175842339aa3b491adeb4015206cda272ff0
Successfully built progressbar
     |████████████████████████████████| 102kB 2.6MB/s 
     |████████████████████████████████| 911kB 7.1MB/s 
  Created wheel for splinter: filename=splinter-0.13.0-cp36-none-any.whl size=33304 sha256=ff4d7fdbe3b25a605b56521d3429ef1e9966e653507b8b36ff406f3876273432
  Stored in directory: /root/.cache/pip/wheels/fd/ff/26/2fc56897bcbe58908bbb2002c53affb516e5e6daa425aefdef
Successfully built splinter


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving ArticlesApril2017.csv to ArticlesApril2017.csv
Saving ArticlesApril2018.csv to ArticlesApril2018.csv
Saving ArticlesFeb2017.csv to ArticlesFeb2017.csv
Saving ArticlesFeb2018.csv to ArticlesFeb2018.csv
Saving ArticlesJan2017.csv to ArticlesJan2017.csv
Saving ArticlesJan2018.csv to ArticlesJan2018.csv
Saving ArticlesMarch2017.csv to ArticlesMarch2017.csv


In [ ]:
from google.colab import files
uploaded2= files.upload()

Saving ArticlesApril2018.csv to ArticlesApril2018 (1).csv


In [ ]:
from google.colab import files
uploaded3 = files.upload()

Saving ArticlesFeb2017.csv to ArticlesFeb2017 (1).csv


In [ ]:
from google.colab import files
uploaded4= files.upload()

Saving ArticlesFeb2018.csv to ArticlesFeb2018.csv


In [ ]:
from google.colab import files
uploaded5= files.upload()

Saving ArticlesJan2017.csv to ArticlesJan2017.csv


In [ ]:
from google.colab import files
uploaded6 = files.upload()

Saving ArticlesJan2018.csv to ArticlesJan2018.csv


In [ ]:
def clean_text(text):
    """ Cleans a text. """
 
    text = text.replace("\t", " ")
    text = text.replace("\n", " ")
    text = text.replace("।", " ")
    text = text.replace("0", " ")
    text = text.replace("1", " ")
    text = text.replace("2", " ")
    text = text.replace("3", " ")
    text = text.replace("4", " ")
    text = text.replace("5", " ")
    text = text.replace("6", " ")
    text = text.replace("7", " ")
    text = text.replace("8", " ")
    text = text.replace("9", " ")
    text = " ".join(text.split())
    return text

In [ ]:
import pandas as pd
corpus = []
for name in uploaded.keys():
  article_df = pd.read_csv(name)
  corpus.extend(list(article_df.headline.values))

In [ ]:
corpus = [h for h in corpus if h != "Unknown"]
len(corpus)

8603

In [ ]:
def clean_text(txt):
    txt = "".join(v for v in txt if v not in string.punctuation).lower()
    txt = txt.encode("utf8").decode("ascii",'ignore')
    return txt 

corpus = [clean_text(x) for x in corpus]
corpus[:10]

['finding an expansive view  of a forgotten people in niger',
 'and now  the dreaded trump curse',
 'venezuelas descent into dictatorship',
 'stain permeates basketball blue blood',
 'taking things for granted',
 'the caged beast awakens',
 'an everunfolding story',
 'oreilly thrives as settlements add up',
 'mouse infestation',
 'divide in gop now threatens trump tax plan']

In [ ]:
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer()

In [ ]:
def get_sequence_of_tokens(corpus):
   
    tokenizer.fit_on_texts(corpus)
    
    total_words = len(tokenizer.word_index) + 1
    
    
    input_sequences = []
    for line in corpus:
        token_list = tokenizer.texts_to_sequences([line])[0]
        
        for i in range(1, len(token_list)):
            n_gram_sequence = token_list[:i+1]
            input_sequences.append(n_gram_sequence)
    return input_sequences, total_words

In [ ]:
input_seq, total_words = get_sequence_of_tokens(corpus)

In [ ]:
input_seq[:10], total_words

([[391, 17],
  [391, 17, 5166],
  [391, 17, 5166, 523],
  [391, 17, 5166, 523, 4],
  [391, 17, 5166, 523, 4, 2],
  [391, 17, 5166, 523, 4, 2, 1601],
  [391, 17, 5166, 523, 4, 2, 1601, 134],
  [391, 17, 5166, 523, 4, 2, 1601, 134, 5],
  [391, 17, 5166, 523, 4, 2, 1601, 134, 5, 1951],
  [7, 57]],
 11265)

In [ ]:
def generate_padded_sequences(input_sequences):
    max_sequence_len = max([len(x) for x in input_sequences])
    input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))
    
    predictors, label = input_sequences[:,:-1],input_sequences[:,-1]
    #print(predictors, label)
    label = ku.to_categorical(label, num_classes=total_words)
    
    return predictors, label, max_sequence_len

In [ ]:
def create_model(max_sequence_len, total_words):
    input_len = max_sequence_len - 1
    model = Sequential()
    
    
    model.add(Embedding(total_words, 10, input_length=input_len))
    
    
    model.add(LSTM(700))
    model.add(Dropout(0.2))
    
   
    model.add(Dense(total_words, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam')
    
    return model

In [ ]:
predictors, label, max_sequence_len = generate_padded_sequences(input_seq)

In [ ]:
model = create_model(max_sequence_len, total_words)
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 23, 10)            112650    
_________________________________________________________________
lstm_2 (LSTM)                (None, 700)               1990800   
_________________________________________________________________
dropout_2 (Dropout)          (None, 700)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 11265)             7896765   
Total params: 10,000,215
Trainable params: 10,000,215
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(predictors, label, epochs=50, verbose=1, batch_size = 64)

Epoch 1/50
51770/51770 [==============================] - 38s 732us/step - loss: 7.7864
Epoch 2/50
51770/51770 [==============================] - 39s 758us/step - loss: 7.3730
Epoch 3/50
51770/51770 [==============================] - 39s 752us/step - loss: 7.1170
Epoch 4/50
51770/51770 [==============================] - 39s 762us/step - loss: 6.8455
Epoch 5/50
51770/51770 [==============================] - 40s 765us/step - loss: 6.5015
Epoch 6/50
51770/51770 [==============================] - 40s 780us/step - loss: 6.0816
Epoch 7/50
51770/51770 [==============================] - 40s 769us/step - loss: 5.5975
Epoch 8/50
51770/51770 [==============================] - 40s 765us/step - loss: 5.1060
Epoch 9/50
51770/51770 [==============================] - 40s 779us/step - loss: 4.6298
Epoch 10/50
51770/51770 [==============================] - 40s 769us/step - loss: 4.2081
Epoch 11/50
51770/51770 [==============================] - 39s 753us/step - loss: 3.8394
Epoch 12/50
51770/51770 [=====

In [ ]:
def generate_text(seed_text, next_words, model, max_sequence_len):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted = model.predict_classes(token_list, verbose=0)
        
        output_word = ""
        for word,index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " " + output_word
    return seed_text.title()

In [ ]:
print (generate_text("United States of", 1, model, max_sequence_len))
print (generate_text("Modi is ", 3, model, max_sequence_len))
print (generate_text("Cricket is", 3, model, max_sequence_len))
print (generate_text("india and china", 2, model, max_sequence_len))
print (generate_text("new york", 2, model, max_sequence_len))
print (generate_text("how to prepare A soaring", 5, model, max_sequence_len))

United States Of Plan
Modi Is  It Ok For
Cricket Is The Blackboard Circuit
India And China Or Talk
New York Today A
How To Prepare A Soaring New Price Story Wins Fashion
